# GIS 관련 API에 대한 도움글

## 1. API란?
API(Application Programming Interface, 응용 프로그램 프로그래밍 인터페이스) : 응용 프로그램에서 사용할 수 있도록, 운영 체제나 프로그래밍 언어가 제공하는 기능을 제어할 수 있게 만든 인터페이스를 뜻한다.<br/>
> 출처 : https://ko.wikipedia.org/wiki/APIs

![Oops](http://blog.wishket.com/wp-content/uploads/2019/10/API-%EC%89%BD%EA%B2%8C-%EC%95%8C%EC%95%84%EB%B3%B4%EA%B8%B0.png, "API 프로세스")<br/>
> 출처 : http://blog.wishket.com/api%EB%9E%80-%EC%89%BD%EA%B2%8C-%EC%84%A4%EB%AA%85-%EA%B7%B8%EB%A6%B0%ED%81%B4%EB%9D%BC%EC%9D%B4%EC%96%B8%ED%8A%B8/

### A. [Naver MAP API](https://www.ncloud.com/product/applicationService/maps) 
[네이버](https://www.ncloud.com/product)에서는 Storage, DB, 이미지, 자연어 등 다양한 API 서비스를 제공하고 있습니다. <br/>
그 중에서 GIS 관련 서비스를 모아둔 MAPS 의 API에 대해서 살펴보겠습니다. <br/>
위의 링크로 가게되면 서비스의 종류와 요금 안내를 받을 수 있습니다. 서비스를 상용화하지 않는다면 대부분 무료로 사용할 수 있다고 생각하시면 됩니다. <br/>
자세한 내용은 [NAVER API 참조서](https://api.ncloud-docs.com/docs/home)를 참고하시기 바랍니다.

---
일단 API를 사용하기 위해서는 [네이버 개발자 센터](https://developers.naver.com/main/)에 회원가입을 한 뒤<br/>
[어플리케이션 등록](https://developers.naver.com/docs/common/openapiguide/appregister.md#%EC%95%A0%ED%94%8C%EB%A6%AC%EC%BC%80%EC%9D%B4%EC%85%98-%EB%93%B1%EB%A1%9D)을 하여 api key를 받아야합니다. <br/>
저는 사전에 받은 api key를 불러오도록 하겠습니다.

In [ ]:
import os
import sys


import pandas as pd

# API를 사용하기 위한 라이브러리
import requests
import json

# 데이터 시각화를 위한 라이브러리
import folium
import matplotlib
%matplotlib inline

In [ ]:
key_csv = pd.read_csv("key/api_key.secret", index_col = "name")

위의 링크에 걸려있는 API 참조서에서 geocoding 과 reverse geocoding을 해보겠습니다.<br/>
**geocoding 은 주소 -> 좌표**<br/>
**reverse geocoding 은 좌표 -> 주소 변환을 해주는 역할을 합니다.**<br/>
<br/>
---
참조서에서 request의 방식, header에 넣어야될 파라미터 값 그리고 요청 파라미터를 어떤 이름으로 어떻게 넣어야되는지 나와있습니다.<br/>
예시를 위해서 서울 시청 주소를 geocoding 하여 좌표로 바꿔보도록 하겠습니다.

In [ ]:
# 위에서 말했던 header에 넣어야할 id와 key를 넣어주고 url 을 지정해줍니다.
header = {}
header["Accept"]="application/json"
header["X-NCP-APIGW-API-KEY-ID"] = key_csv.loc["naver_id"]["value"]
header["X-NCP-APIGW-API-KEY"] = key_csv.loc["naver_key"]["value"]

In [ ]:
naver_geocoding = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
# 서울시청 주소
exam_address = "서울특별시 중구 세종대로 110"

In [ ]:
# get 방식으로하기 위해 url 뒤에 ?를 붙이면 어떠한 데이터를 보내겠다는 이야기입니다. 그러므로 query 뒤에 주소를 하나 적어서 보내봅니다.
naver_geturl = naver_geocoding + "?query="+ str(exam_address)
print("naver_geturl = {}".format(naver_geturl))

In [ ]:
res = requests.get(naver_geturl, headers = header)
res_ = json.loads(res.text)
res_

---
참조서에서 보시는 **응답바디**라는 부분과 똑같이 리턴을 받을 수 있게됩니다.
아주 친절하게도 그냥 좌표만 나오는 것이 아니라 행정동과 건물의 이름 구주소 등도 알 수 있습니다.
---

In [ ]:
# python의 folium으로 띄워보겠습니다.
m = folium.Map(
    location=[res_["addresses"][0]["y"],res_["addresses"][0]["x"]],
    zoom_start=18
)

folium.Marker(
  location=[res_["addresses"][0]["y"],res_["addresses"][0]["x"]],
  popup=res_["addresses"][0]["addressElements"][6]["longName"],
  icon=folium.Icon(color='red',icon='star')
).add_to(m)

m

---
다시 **reverse geocoding**을 통하여 좌표가 다시 서울시청 주소로 나오는지 확인해보겠습니다.<br/>
reverse geocoding의 경우 출력 format을 요청 파라미터로 지정해주고, 변환 주소 타입이 지번 혹은 도로명인지에 대해서도 지정을 해주어야합니다. <br/>
요청 파라미터에 도로명 주소와 json 형식으로 받아오도록 request를 해보겠습니다.

---

In [ ]:
naver_geocoding = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc"
exam_point = ",".join([res_["addresses"][0]["x"], res_["addresses"][0]["y"]])
output_format = "json"

In [ ]:
naver_geturl = f"{naver_geocoding}?coords={str(exam_point)}&orders=roadaddr&output={output_format}"
print("naver_geturl = {}".format(naver_geturl))

In [ ]:
res = requests.get(naver_geturl, headers = header)
res_ = json.loads(res.text)
res_

In [ ]:
print(" ".join([res_["results"][0]["region"][f"area{i + 1}"]["name"] for i in range(3)] + [res_["results"][0]["land"]["number1"]]))
print(exam_address)

---
결과가 조금 이상하게 나왔지만
[세종대로? 태평대로?](https://dorojuso.kr/1114010300100310000019224/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C-%EC%A4%91%EA%B5%AC-%EC%84%B8%EC%A2%85%EB%8C%80%EB%A1%9C-110-%ED%83%9C%ED%8F%89%EB%A1%9C1%EA%B0%80) <br/>
잘 맞게 나왔다는 것을 알 수 있습니다.

---

---
### **B. [Kakao MAP API](https://apis.map.kakao.com/web/)**
카카오는 네이버에서 제공하는 모든 API를 모아둔 사이트가 있는 것과 달리 Kakao Map API 페이지가 따로 있습니다.<br/>
API가 익숙하신 분들이라면 크게 상관 없겠지만 왠지 카카오가 좀 더 친절하다고 느껴집니다.&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*~~네이버도 예전에는 Map API 사이트가 따로 있었고 사이트 구조도 비슷했었습니다.~~* <br/>
왜 더 친절하게 느껴지는 지는 [Kakao MAP API Sampler](https://apis.map.kakao.com/web/sample/) 여기에 들어가면 샘플 코드로 html과 js로 실행할 수 있는 샘플들도 할 수 있습니다.

---


#### **<u>HTML, CSS, JS 아무것도 모르는 데이터 사이언티스트인데 이걸 왜 쓰고 어떻게 쓰나요?</u>**
물론 웹 관련하여 지식이 없는 분들에겐 어렵고 복잡하게만 느껴지시겠지만, <br/>
굳이 지금 당장 파이썬을 안키고 자바스크립트에 간단하게 값만 넣으면 샘플로 효율적으로 작업하실 수 있습니다.

---
카카오도 다른 API들과 마찬가지로 회원가입과 어플리케이션 등록 후 key를 발급 받아야 사용할 수 있습니다.

---
Kakao MAP API 실습은 네이버에서는 볼 수 없었던 **키워드로 검색**하는 기능을 해보겠습니다.

---
### C. [T map API]() 


In [ ]:
headers = {}
headers["appKey"] = key_csv.loc["tmap_key"]["value"]
headers["Content-Type"] = "application/json"
param = {}

In [ ]:
# 여기서 좌표값을 string 타입으로 바꿔주지 않으면, api자체에서 type error가 뜹니다. 주의해서 string으로 바꿔주시길 바랍니다.
param["startName"] = tmp.iloc[0]["장소"]
param["startX"] = str(tmp.iloc[0]["X_open"])
param["startY"] = str(tmp.iloc[0]["Y_open"])
param["startTime"] = "202002210900"

In [ ]:
param["endName"] = tmp.iloc[4]["장소"]
param["endX"] = str(tmp.iloc[4]["X_open"])
param["endY"] = str(tmp.iloc[4]["Y_open"])

In [ ]:
param

In [ ]:
viaPoints = [{"viaPointId" : str(a), "viaPointName" : tmp.iloc[a]["장소"] , "viaX" : str(tmp.iloc[a]["X_open"]), "viaY" : str(tmp.iloc[a]["Y_open"])} for a in range(1,4)]

In [ ]:
viaPoints

In [ ]:
param["viaPoints"] = viaPoints

In [ ]:
param

In [ ]:
param["reqCoordType"] = "WGS84GEO"
param["resCoordType"] = "WGS84GEO"

In [ ]:
res = requests.post(url = "https://apis.openapi.sk.com/tmap/routes/routeSequential30?version=1&format=json", data = json.dumps(param), headers = headers)

In [ ]:
res = res.json()

In [ ]:
# 경유지가 총 5개 이므로, line string 값이 4개가 나왔습니다. 이에 대한 index는 2~ 짝수의 인덱스에 들어가있습니다.
res["features"][2]["geometry"]

In [ ]:
import geopandas as gpd
from shapely.geometry import LineString

In [ ]:
# 경유지가 총 5개 이므로, line string 값이 4개가 나왔습니다. 이에 대한 index는 2~ 짝수의 인덱스에 들어가있습니다.
LineString(res["features"][2]["geometry"]["coordinates"])

In [ ]:
df = pd.DataFrame([[res["features"][idx]["properties"]["viaPointName"], res["features"][idx]["geometry"]["coordinates"]] for idx in range(len(res["features"])) if idx % 2 != 0 ], columns = ["name", "point"])

In [ ]:
res["features"][2]["geometry"]["coordinates"][1:10]

In [ ]:
df["geometry"] = pd.NA

In [ ]:
df.head()

In [ ]:
for idx in range(1, len(res["features"])) :
    if idx % 2 == 0 :
        df["geometry"][idx/2-1] = LineString(res["features"][idx]["geometry"]["coordinates"])

In [ ]:
df.head()

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry = "geometry")

In [ ]:
gdf.head()

In [ ]:
# 결과가 잘 나왔는지 r의 리플랫과 비슷한 python의 folium으로 띄워보겠습니다.
m = folium.Map(
    location=[gdf["point"][0][1],gdf["point"][0][0]],
    zoom_start=13
)

for idx in range(len(gdf)) :
    folium.Marker(
      location=[gdf["point"][idx][1],gdf["point"][idx][0]],
      popup=gdf["name"][idx],
      icon=folium.Icon(color='red',icon='star')
    ).add_to(m)
m

In [ ]:
folium.GeoJson(data=gdf.to_json()).add_to(m)
m

In [ ]:
import random 

In [ ]:
random.uniform(126, 127)

In [ ]:
# 과연 몇개까지 렌더링이 되는지 테스트 해보겠습니다.
m = folium.Map(
    location=[gdf["point"][0][1],gdf["point"][0][0]],
    zoom_start=10
)

for idx in range(5000) :
    folium.Marker(
      location=[random.uniform(35, 38), random.uniform(126, 128)],
      icon=folium.Icon(color='red',icon='star')
    ).add_to(m)
m

### <font color = "red"> 단점은 여러가지 있습니다. </font>
#### 일단은 for문으로 하나하나 add 시켜줘야한다는 점이 맘에 안드는군요...~~개인적으로 for문을 선호하지 않습니다.~~
#### 그리고 일정 수 이상 늘어나면 렌더링이 되지 않는 현상이 발생합니다.
##### 이거에 관해서는 말이 많습니다. 어떤 분들은 훨씬 더 많이 띄워도 나온다는 분들이 계시는데, 안되는 사람은 꼭 안되요...
##### 그리고 geopandas 같은 라이브러리와 호환성이 그닥 좋지 않습니다... leaflet과 sf 패키지 처럼... 
##### 마지막으로... 안 이뻐요.... 

In [ ]:
Image("../semina/folium_limit.jpg")

## 좀 더 고급 공간 시각화 툴인 pydeck이란 것을 찾을 수 있었습니다!

In [ ]:
import pydeck as pdk
# import pandas as pd
# import geopandas as gpd
# import random
# key_csv = pd.read_csv("../semina/api_key.csv", index_col = "name")
# MAPBOX_API_KEY = key_csv.loc["mapbox_key"]["value"]

In [ ]:
Image("../semina/pydeck_setting.jpg")

## 주피터랩의 경우에는 이것을 셋팅하고 별도의 labextension 을 설치해주어야합니다.

In [ ]:
Image("../semina/pydeck_intro.jpg")

### pydeck의 주요 특징입니다.
#### 다양한 장점이 있습니다.
#### 첫째로는 GPU 계산을 통해 정확성과 속도가 높다는 점이 있겠습니다. 그로 인해서 복잡한 레이어도 쉽게 패키징할 수 있습니다.
#### 둘째로는 이쁜 반응형 지도를 만들 수 있습니다.
#### 마지막으로 장점이자 단점은, 다른 API들은 javascript 기반으로 되어있어 시각화가 힘들지만, pydeck을 활용하면 jupyter에서 바로 시각화할 수 있습니다. 하지만, jupyter를 위해 개발되었기 때문에, 아마 pycharm 등의 다른 IDE에서는 구현되지 않을 가능성도 있습니다.

In [ ]:
sig_shape = gpd.read_file("../data/sig_shape.shp", encoding = "cp949")

In [ ]:
sig_shape["total_sum"] = pd.NA
sig_shape["total_sum"] = sig_shape["total_sum"].apply(lambda x : random.random()*10000)

In [ ]:
OPENMATE_KATEC = "+proj=tmerc +lat_0=38 +lon_0=128 +k=0.9999 +x_0=400000 +y_0=600000 +ellps=bessel +towgs84=-115.8,474.99,674.11,1.16,-2.31,-1.63,6.43 +units=m +no_defs"
OPENMATE_WGS =  "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs +towgs84=0,0,0"
OPENMATE_UTM = "+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs"

In [ ]:
# sig_shape.to_crs(crs = OPENMATE_KATEC)

# # 7분 30초가 걸립니다... geopandas 개 구리네요..
# %time sig_shape = sig_shape.to_crs(crs = OPENMATE_WGS)

In [ ]:
sig_shape.head()

In [ ]:
# 폴리곤 형태를 받지 못하기 때문에, 이것을 리스트로 바꿔주는 함수를 만들어 실행하겠습니다.
def multipolygon_to_coordinates(x): 
    try:
        lon, lat = x.exterior.xy
    except :
        lon, lat = x[0].exterior.xy
    return [[x, y] for x, y in zip(lon, lat)] 

In [ ]:
sig_shape["coordinates"] = sig_shape['geometry'].apply(multipolygon_to_coordinates)

In [ ]:
sig_shape.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
# 색깔을 나타내기 위해서 minmax scale을 하고 255를 곱해주겠습니다.
mmscaler = MinMaxScaler()
mmscaler.fit(sig_shape.filter(regex = "total_sum"))
sig_shape["mm_scaler"] = mmscaler.transform(sig_shape.filter(regex = "total_sum"))

In [ ]:
del sig_shape["geometry"]

In [ ]:
sig_shape = pd.DataFrame(sig_shape)

In [ ]:
# get_fill_color의 리스트는 각각 rgb를 뜻하며, 저는 초록색에 minmax 스케일한 값을 곱해주겠습니다.
layer = pdk.Layer( 'PolygonLayer', 
                  sig_shape, 
                  get_polygon='coordinates', 
                  get_fill_color='[0, 255*mm_scaler, 0]', 
                  pickable=True,
                  get_elevation = 'total_sum',
                  auto_highlight=True,
                  opacity = 0.8
                 )

In [ ]:
center = [126.986, 37.565] 
view_state = pdk.ViewState( longitude=center[0], latitude=center[1], zoom=5)

## 이쁜가요???
### ~~여기까지 오기 너무 힘들었어요..~~

In [ ]:
# 일단 2D로 나타낸 지도입니다.
# 중간중간 빠진 시군구가 있는데... 왜 그런지는 잘 모르겠습니다...
r = pdk.Deck(layers=[layer], initial_view_state=view_state) 
r.to_html("../semina/pydeck_test1.html")
# r.show()

In [ ]:
# 이번엔 3D로 rendering해보겠습니다.
# 높이를 아까 random으로 주었던 값으로 해보겠습니다.
layer.extruded = True; 
# layer.get_elevation = 'total_sum'
layer.elevation_scale = 1

view_state.bearing=15
view_state.pitch=45

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html("../semina/pydeck_test2.html")
# r.show()

In [ ]:
import geojson as geojson

In [ ]:
sig_shape = gpd.read_file("../data/sig_shape.shp", encoding = "cp949")

In [ ]:
sig_shape["total_sum"] = sig_shape.apply(lambda x : random.random(), axis = 1)

In [ ]:
# get_fill_color의 리스트는 각각 rgb를 뜻하며, 저는 초록색에 minmax 스케일한 값을 곱해주겠습니다.
LAND_COVER = [[[-123.0, 49.196], [-123.0, 49.324], [-123.306, 49.324], [-123.306, 49.196]]]

polygon_layer = pdk.Layer(
    'PolygonLayer',
    LAND_COVER,
    stroked=False,
    # processes the data as a flat longitude-latitude pair
    get_polygon='-',
    get_fill_color=[0, 0, 0, 20]
)

geojson_layer = pdk.Layer(
    'GeoJsonLayer',
    geojson.loads(geojson.dumps(sig_shape)),
    opacity=0.8,
    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    get_fill_color='[255, 255, 255 * properties.total_sum]',
    get_line_color=[255, 255, 255],
    pickable=True
)
center = [126.986, 37.565]
view_state = pdk.ViewState( longitude=center[0], latitude=center[1], zoom=5)

r = pdk.Deck(layers=[polygon_layer, geojson_layer], initial_view_state=view_state)
r.to_html("../semina/pydeck_test3.html")

## 이 외에 다양한 시각화를 살펴보겠습니다. 예시에 나와있습니다.

In [ ]:
Image("../semina/mapbox_exam1.jpg")

### ~~이러한 지도는 선거 관련 인지도 조사할 때 이쁘게 만들어질 것 같습니다. ~~

In [ ]:
Image("../semina/mapbox_exam2.jpg")

In [ ]:
Image("../semina/mapbox_exam23jpg.jpg")

## ~~이런 시각화는 코로나 맵 만들 때 괜찮을 것 같습니다.~~

In [ ]:
# reference :
# https://deck.gl/#/documentation/deckgl-api-reference/layers/composite-layer
# https://zzsza.github.io/data/2019/11/24/pydeck/
# https://dailyheumsi.tistory.com/147?category=854906
# https://deckgl.readthedocs.io/en/latest/layer.html

# 이상으로 api 소개를 마치겠습니다.
# 감사합니다!